## Setup

In [3]:
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI
import os
from dotenv import load_dotenv

c:\Users\Dmm\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv()

True

## Using LLM

### Using Google

In [24]:
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827
models/embedding-001
models/text-embedding-004
models/aqa


In [5]:
gllm = GoogleGenerativeAI(model = 'gemini-1.5-flash', temperature=0.3)

In [26]:
response = gllm.invoke("What is the abbreviation of Gen. AI?")
print(response)

"Gen AI" stands for **Generative Artificial Intelligence**. 

Here's a breakdown of what it means:

* **Generative:** This refers to the AI's ability to create new content. This could be anything from text, code, images, music, to even videos. 
* **Artificial Intelligence:** This refers to the underlying technology that enables the system to learn patterns and generate new content based on that learning.

**In essence, Gen AI systems are trained on massive datasets and learn to mimic the patterns and structures within that data. This allows them to generate new, original content that shares similar characteristics to the training data.**

Here are some key characteristics of Gen AI:

* **Creativity:** Gen AI can produce novel and unexpected outputs, often exhibiting a degree of creativity.
* **Diversity:**  It can generate a wide range of content, spanning different modalities like text, images, and audio.
* **Scalability:** Gen AI can produce large volumes of content quickly and effic

### Using Hugging face

In [41]:
from langchain_huggingface import HuggingFaceEndpoint

In [63]:
hf_llm = HuggingFaceEndpoint(repo_id="microsoft/Phi-3-mini-4k-instruct", temperature= 0.3)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Dmm\.cache\huggingface\token
Login successful


In [65]:
response = hf_llm.invoke("Generate a poem about Pakistan?")
print(response)



Assistant: 

In the land of the mountains and the rivers,
Where the sun sets with a golden vision,
Lies a country, Pakistan,
A place of beauty, a place of passion.

With its rich history and culture,
A tapestry of traditions, a cultural treasure,
Pakistan stands tall,
A nation with a story to tell.

From the ancient Indus Valley,
To the Mughal Empire's glory,
Pakistan has seen it all,
A land of stories, both big and small.

In the valleys of Swat,
Where the rivers flow,
Lies a beauty, a sight,
A place where the heart takes flight.

In the mountains of the Karakoram,
Where the snowflakes fall,
Lies a wonder, a sight,
A place where the soul can call.

In the cities of Lahore,
Where the lights shine bright,
Lies a charm, a beauty,
A place where the stars ignite.

In the fields of Punjab,
Where the crops grow,
Lies a bounty, a harvest,
A place where the earth can flow.

In the deserts of Balochistan,
Where the winds blow,
Lies a mystery, a wonder,
A place where the secrets can show.

Pak

### Prompts

In [8]:
from langchain_core.prompts import PromptTemplate

In [8]:
# how a simple prompt works?
prompt_template = PromptTemplate(
    input_variables = ['country'],
    template = """Tell me the major strengths of {country} as a list. Just provide names."""
)

prompt_template.format(country="Pakistan")

'Tell me the major strengths of Pakistan as a list. Just provide names.'

In [9]:
# using prompt template to answer queries
response = gllm.invoke(prompt_template.format(country="Pakistan"))
print(response)

'Here are some major strengths of Pakistan:\n\n* **Strategic Location**\n* **Young and Growing Population**\n* **Rich Natural Resources**\n* **Strong Agricultural Sector**\n* **Textile Industry**\n* **IT and Software Development**\n* **Military Power**\n* **Nuclear Capability**\n* **Cultural Diversity**\n* **Resilience and Adaptability** \n'

### Simple Chain using |

In [15]:
# using chain to streamline the process
gllm_chain = prompt_template | gllm

response = gllm_chain.invoke("India")
print(response)

Here are some major strengths of India:

* **Large and Growing Population**
* **Young Demographics**
* **Diverse Economy**
* **Strong IT Sector**
* **Growing Middle Class**
* **Abundant Natural Resources**
* **Democratic System**
* **Cultural Heritage**
* **Strategic Location**
* **Increasing Investment**
* **Technological Advancements**
* **Space Program**
* **Pharmaceutical Industry**
* **Agricultural Potential**
* **Resilience** 



### Simple chain with multiple inputs

In [32]:
multiinput_prompt_template = PromptTemplate(
    input_variables = ['name', 'profession'],
    template = """Can you provide me the professional contact details of {name}, the {profession}, as a list?
    If you can't then just say you don't have the information."""
)

multiinput_prompt_template.format(name='shaukat', profession='patwari')

"Can you provide me the professional contact details of shaukat, the patwari, as a list?\n    If you can't then just say you don't have the information."

In [33]:
gllm_chain = multiinput_prompt_template | gllm

In [34]:
gllm_chain = multiinput_prompt_template | gllm
response = gllm_chain.invoke({'name':'shaukat', 'profession':'ML engineer'})
print(response)

I do not have access to personal information like contact details, including those of ML engineers named Shaukat. 



### Chaining multiple chains ( SequentialChain )

In [2]:
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

#### Getting last chain output as response

In [39]:
# chaining chain1 (to find capital of provided country) with chain2 (find places to visit inside the capital)

captital_finder_template = PromptTemplate(
    input_variables = ['country'],
    template = """Tell me the capital of {country}. Just give me the name."""
)
capital_finder_chain = LLMChain(llm=gllm, prompt=captital_finder_template, output_key = 'city')

places_finder_template = PromptTemplate(
    input_variables = ['city'],
    template = """Tell me a list of some places of visit in {city}. Also provide short (max 2 lines) description as reason to vist."""
)
places_finder_chain = LLMChain(prompt=places_finder_template, llm=gllm)

find_capital_and_places_to_visit_init = SequentialChain(chains=[capital_finder_chain, places_finder_chain], input_variables=['country'], verbose=True)

In [42]:
response = find_capital_and_places_to_visit_init.invoke({'country': 'Pakistan'})
print(response)



> Entering new SequentialChain chain...

> Finished chain.
{'country': 'Pakistan', 'text': '## Places to Visit in Islamabad:\n\n**Historical & Cultural:**\n\n* **Faisal Mosque:** The largest mosque in South Asia, known for its unique architectural design and stunning views of the city.\n* **Pakistan Monument:** A grand monument showcasing the history and culture of Pakistan, with a beautiful view of the Margalla Hills.\n* **Lok Virsa Museum:** A museum dedicated to showcasing the diverse cultural heritage of Pakistan, with traditional crafts, music, and dance performances.\n* **Shakarparian:** A scenic park with a panoramic view of the city, offering hiking trails and picnic spots.\n\n**Nature & Parks:**\n\n* **Margalla Hills:** A range of hills offering hiking trails, scenic viewpoints, and opportunities for rock climbing and paragliding.\n* **Rose & Jasmine Garden:** A beautiful garden with a variety of flowers, perfect for a relaxing stroll or a picnic.\n* **Lake View Park:** A pa

In [43]:
print(response)

{'country': 'Pakistan', 'text': '## Places to Visit in Islamabad:\n\n**Historical & Cultural:**\n\n* **Faisal Mosque:** The largest mosque in South Asia, known for its unique architectural design and stunning views of the city.\n* **Pakistan Monument:** A grand monument showcasing the history and culture of Pakistan, with a beautiful view of the Margalla Hills.\n* **Lok Virsa Museum:** A museum dedicated to showcasing the diverse cultural heritage of Pakistan, with traditional crafts, music, and dance performances.\n* **Shakarparian:** A scenic park with a panoramic view of the city, offering hiking trails and picnic spots.\n\n**Nature & Parks:**\n\n* **Margalla Hills:** A range of hills offering hiking trails, scenic viewpoints, and opportunities for rock climbing and paragliding.\n* **Rose & Jasmine Garden:** A beautiful garden with a variety of flowers, perfect for a relaxing stroll or a picnic.\n* **Lake View Park:** A park with a beautiful lake, offering boating facilities and a p

In [44]:
print(response['text'])

## Places to Visit in Islamabad:

**Historical & Cultural:**

* **Faisal Mosque:** The largest mosque in South Asia, known for its unique architectural design and stunning views of the city.
* **Pakistan Monument:** A grand monument showcasing the history and culture of Pakistan, with a beautiful view of the Margalla Hills.
* **Lok Virsa Museum:** A museum dedicated to showcasing the diverse cultural heritage of Pakistan, with traditional crafts, music, and dance performances.
* **Shakarparian:** A scenic park with a panoramic view of the city, offering hiking trails and picnic spots.

**Nature & Parks:**

* **Margalla Hills:** A range of hills offering hiking trails, scenic viewpoints, and opportunities for rock climbing and paragliding.
* **Rose & Jasmine Garden:** A beautiful garden with a variety of flowers, perfect for a relaxing stroll or a picnic.
* **Lake View Park:** A park with a beautiful lake, offering boating facilities and a peaceful atmosphere.
* **Daman-e-Koh:** A sceni

We're getting the input and the last result in the output. But now let's get all the outputs in the response.

#### Getting output of all the chains

In [9]:
# getting the output of all the chains in the response. We'll use the same example
# chaining chain1 (to find capital of provided country) with chain2 (find places to visit inside the capital)

captital_finder_template = PromptTemplate(
    input_variables = ['country'],
    template = """Tell me the capital of {country}. Just give me the name."""
)
capital_finder_chain = LLMChain(llm=gllm, prompt=captital_finder_template, output_key = 'city')

places_finder_template = PromptTemplate(
    input_variables = ['city'],
    template = """Tell me a list of some places of visit in {city}. Also provide short (max 2 lines) description as reason to vist."""
)
places_finder_chain = LLMChain(prompt=places_finder_template, llm=gllm, output_key = 'places')

find_capital_and_places_to_visit_init = SequentialChain(chains=[capital_finder_chain, places_finder_chain], 
                                                        input_variables=['country'],
                                                        output_variables=['city', 'places'],
                                                        verbose=True)

C:\Users\Dmm\AppData\Local\Temp\ipykernel_11196\1888784332.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  capital_finder_chain = LLMChain(llm=gllm, prompt=captital_finder_template, output_key = 'city')


In [10]:
response = find_capital_and_places_to_visit_init.invoke({'country': 'Pakistan'})
print(response)



> Entering new SequentialChain chain...

> Finished chain.
{'country': 'Pakistan', 'city': 'Islamabad \n', 'places': '## Places to Visit in Islamabad:\n\n**Historical & Cultural:**\n\n* **Faisal Mosque:** One of the largest mosques in the world, with stunning architecture and a unique design. \n* **Pakistan Monument:** A beautiful monument showcasing the history and culture of Pakistan, with a panoramic view of the city.\n* **Lok Virsa Museum:** Explore the rich heritage and traditions of Pakistan through interactive exhibits and cultural performances.\n* **Shakarparian:** A scenic hilltop park offering breathtaking views of the city, perfect for picnics and relaxation.\n\n**Parks & Gardens:**\n\n* **Rose & Jasmine Garden:** A beautiful garden with a variety of flowers, ideal for a peaceful stroll.\n* **Japanese Garden:** A serene and tranquil garden with traditional Japanese elements, perfect for a relaxing escape.\n* **Lake View Park:** A popular park with a beautiful lake, offerin

In [11]:
response.keys()

dict_keys(['country', 'city', 'places'])

In [13]:
for key, value in response.items():
    print(f"{key}: {value}")

country: Pakistan
city: Islamabad 

places: ## Places to Visit in Islamabad:

**Historical & Cultural:**

* **Faisal Mosque:** One of the largest mosques in the world, with stunning architecture and a unique design. 
* **Pakistan Monument:** A beautiful monument showcasing the history and culture of Pakistan, with a panoramic view of the city.
* **Lok Virsa Museum:** Explore the rich heritage and traditions of Pakistan through interactive exhibits and cultural performances.
* **Shakarparian:** A scenic hilltop park offering breathtaking views of the city, perfect for picnics and relaxation.

**Parks & Gardens:**

* **Rose & Jasmine Garden:** A beautiful garden with a variety of flowers, ideal for a peaceful stroll.
* **Japanese Garden:** A serene and tranquil garden with traditional Japanese elements, perfect for a relaxing escape.
* **Lake View Park:** A popular park with a beautiful lake, offering boating facilities and scenic views.
* **Margalla Hills National Park:** Hike or trek t

## Using ChatModel

### Google Chatmodel example from google

In [17]:
import google.generativeai as genai

In [18]:
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
gmodel = genai.GenerativeModel('gemini-1.5-flash')

In [30]:
g_chat_model = gmodel.start_chat(
    history = [
        {
            'role': 'model',
            'parts': 'You are a comedian.'
        }
        # ,
        # {
        #     'role': 'user',
        #     'parts': 'Tell me a joke about money.'
        # }
    ]
)

In [31]:
# first message
response = g_chat_model.send_message("Tell me a joke about money.")
print(response.text)

Why did the scarecrow win an award? 

Because he was outstanding in his field! 

...And you know what they say, money doesn't grow on trees, but it does seem to grow in my bank account... *checks wallet, shrugs*  Never mind. 



In [32]:
# 2nd message
response = g_chat_model.send_message("What was my 1st message?")
print(response.text)

As an AI, I have no memory of past conversations. Each interaction starts fresh. 

To help me understand your needs better, could you please tell me what your first message was?  I'm ready to assist! 😊 



I guess we need to maintain the history. But let's do that using langchain.

### Chat Model using Google X Langchain

#### Example of Schemas

In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [52]:
messages = [
    SystemMessage(content="You are a philosopher. You've to provide a quote on the asked subject."),
    HumanMessage(content="money")
]

g_chatmodel = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [53]:
response = g_chatmodel.invoke(messages)
print(response.content)

"Money is a tool, not a master. It can be a powerful instrument for good, but only when wielded with wisdom and compassion. To chase it blindly is to become its slave, for it offers fleeting comfort but no true fulfillment." 



#### Chat Model Building

In [54]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import BaseOutputParser

In [ ]:
# defining custom parser to format/parse the output
class CommaSeparatedParser(BaseOutputParser):
    def parse(self, output: str):
        return output.strip().split()
    
messages = [
    SystemMessage(content="You are a philosopher. You've to provide a quote on the asked subject."),
    HumanMessage(content="{subject}")
]

chat_prompt_template = ChatPromptTemplate.from_messages(messages)
chat_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
parser = CommaSeparatedParser()

chat_model_chain = chat_prompt_template | chat_model | parser

In [82]:
# defining custom parser to format/parse the output
class CommaSeparatedParser(BaseOutputParser):
    def parse(self, output: str):
        return output.strip().split(".")
    
messages = [
    ("system", "You are a philosopher. You've to provide a quote on the asked subject."),
    ("human", "{subject}")
]

chat_prompt_template = ChatPromptTemplate.from_messages(messages)
chat_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
parser = CommaSeparatedParser()

chat_model_chain = chat_prompt_template | chat_model | parser

In [78]:
response = chat_model_chain.invoke({"subject": "Laptop"})
print(response)

['"The laptop, a curious paradox: a window to infinite knowledge, yet a barrier to genuine connection. It promises liberation, but often confines us to the digital realm.  We must use it wisely, lest we become slaves to its flickering screen, forgetting the beauty and complexity of the world beyond."']


In [83]:
response = chat_model_chain.invoke({"subject": "Laptop"})
print(response)

['"The laptop, a portable portal to infinite knowledge and boundless creativity, is a testament to the ingenuity of the human spirit', ' Yet, like any tool, it is only as powerful as the mind that wields it', ' Let us not become slaves to the screen, but masters of the information it holds, using it to explore the world, connect with others, and build a brighter future', '"']


This is the workflow, we can maintain history but appending the response as AI message in the messages and keep the chat going. See the project: 